Source- http://econometricsense.blogspot.ca/2011/11/regression-via-gradient-descent-in-r.html
[_cache_](https://drive.google.com/file/d/0B0J1O2jMMERWZWRHNnJfQnZHdXc/view?usp=drivesdk)

In [30]:
# PROGRAM NAME: gradient_descent_OLS_R
# DATE: 11/27/11
# CREATED BY: MATT BOGARD 
# ----------------------------------------------------------------------------------
#  PURPOSE: illustration of gradient descent algorithm applied to OLS
#  REFERENCE: adapted from : http://www.cs.colostate.edu/~anderson/cs545/Lectures/week6day2/week6day2.pdf                
#   and http://www.statalgo.com/2011/10/17/stanford-ml-1-2-gradient-descent/

In [1]:
rm(list = ls(all = TRUE)); ls() # make sure previous work is clear

character(0)

In [2]:
x0 <- c(1,1,1,1,1) # column of 1's
x1 <- c(1,2,3,4,5) # original x-values

In [3]:
# create the x- matrix of explanatory variables
x <- as.matrix(cbind(x0,x1))
# create the y-matrix of dependent variables 
y <- as.matrix(c(3,7,5,11,14))
m <- nrow(y)

In [12]:
x
y
m

x0,x1
1,1
1,2
1,3
1,4
1,5


3
7
5
11
14


[1] 5

In [13]:
# implement feature scaling
x.scaled <- x
x.scaled[,2] <- (x[,2] - mean(x[,2]))/sd(x[,2])
x.scaled

x0,x1
1.000000,-1.264911
1.0000000,-0.6324555
1,0
1.0000000,0.6324555
1.000000,1.264911


In [15]:
# Analytical results with matrix algebra
# Basic regression coefficient estimation in matrix algebra b = (x'x)^-1 * y. (using the raw x-values).

solve(t(x)%*%x) %*% t(x) %*% y # w/o feature scaling

x0,0.2
x1,2.6


In [16]:
solve(t(x.scaled)%*%x.scaled)%*%t(x.scaled)%*%y # w/ feature scaling

x0,8
x1,4.110961


In [17]:
# results using canned lm function match results above
summary(lm(y ~ x[, 2])) # w/o feature scaling


Call:
lm(formula = y ~ x[, 2])

Residuals:
   1    2    3    4    5 
 0.2  1.6 -3.0  0.4  0.8 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)  
(Intercept)   0.2000     2.1323   0.094   0.9312  
x[, 2]        2.6000     0.6429   4.044   0.0272 *
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 2.033 on 3 degrees of freedom
Multiple R-squared:  0.845,	Adjusted R-squared:  0.7933 
F-statistic: 16.35 on 1 and 3 DF,  p-value: 0.02721


In [18]:
summary(lm(y ~ x.scaled[, 2])) # w/feature scaling


Call:
lm(formula = y ~ x.scaled[, 2])

Residuals:
   1    2    3    4    5 
 0.2  1.6 -3.0  0.4  0.8 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)   
(Intercept)     8.0000     0.9092   8.799  0.00309 **
x.scaled[, 2]   4.1110     1.0165   4.044  0.02721 * 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 2.033 on 3 degrees of freedom
Multiple R-squared:  0.845,	Adjusted R-squared:  0.7933 
F-statistic: 16.35 on 1 and 3 DF,  p-value: 0.02721


In [19]:
# Define the gradient function dJ/dtheata: 1/m * (h(x)-y))*x where h(x) = x*theta
# in matrix form this is as follows:
grad <- function(x, y, theta) {
  gradient <- (1/m)* (t(x) %*% ((x %*% t(theta)) - y))
  return(t(gradient))
} 

In [20]:
# define gradient descent update algorithm
grad.descent <- function(x, maxit){
    theta <- matrix(c(0, 0), nrow=1) # Initialize the parameters
 
    alpha = .05 # set learning rate
    for (i in 1:maxit) {
      theta <- theta - alpha  * grad(x, y, theta)   
    }
 return(theta)
}

In [21]:
# results without feature scaling
print(grad.descent(x,1000))

            x0       x1
[1,] 0.2000994 2.599972


In [22]:
# results with feature scaling
print(grad.descent(x.scaled,1000))

     x0       x1
[1,]  8 4.110961


In [23]:
# ----------------------------------------------------------------------- 
# cost and convergence intuition
# -----------------------------------------------------------------------
 
# typically we would iterate the algorithm above until the 
# change in the cost function (as a result of the updated b0 and b1 values)
# was extremely small value 'c'. C would be referred to as the set 'convergence'
# criteria. If C is not met after a given # of iterations, you can increase the
# iterations or change the learning rate 'alpha' to speed up convergence

In [25]:
# get results from gradient descent
beta <- grad.descent(x,1000)
beta

x0,x1
0.2000994,2.5999725


In [26]:
# define the 'hypothesis function'
h <- function(xi,b0,b1) {
 b0 + b1 * xi 
}

In [27]:
# define the cost function   
cost <- t(mat.or.vec(1,m))
  for(i in 1:m) {
    cost[i,1] <-  (1 /(2*m)) * (h(x[i,2],beta[1,1],beta[1,2])- y[i,])^2 
  }
 
totalCost <- colSums(cost)
print(totalCost) 

[1] 1.24


In [28]:
# save this as Cost1000
cost1000 <- totalCost

In [29]:
# change iterations to 1001 and compute cost1001
beta <- (grad.descent(x,1001))
cost <- t(mat.or.vec(1,m))
  for(i in 1:m) {
    cost[i,1] <-  (1 /(2*m)) * (h(x[i,2],beta[1,1],beta[1,2])- y[i,])^2 
  }
cost1001 <- colSums(cost)
 
# does this difference meet your convergence criteria? 
print(cost1000 - cost1001)

[1] 1.515144e-11


TODO: Add the equations which we are triyng to implement here